In [27]:
%pylab
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import seaborn as sns
sns.set()

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os.path
import scipy as sp
import scipy.stats
import statsmodels as smr
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
scipy.stats.chisqprob = lambda chisq, df: scipy.stats.chi2.sf(chisq, df)

In [39]:
class Project:   
    def __init__(self):
        self.data = None
        self.train_data = None
        self.test_data = None
            
    def _load_data_(self,filepath):
      
        if (~os.path.isfile('train.csv')) | (~os.path.isfile('test.csv')):
            self.data = pd.read_csv(filepath,encoding='ISO-8859-1')
            self.train_test_split()
            self._write_data_()
            
        self.data = pd.read_csv('train.csv')
                                #,encoding='ISO-8859-1')
        self.test_data = pd.read_csv('test.csv',encoding='ISO-8859-1')
        return self.data
        
    def _write_data_(self):
        self.train_data.to_csv('train.csv')
        self.test_data.to_csv('test.csv')
    
    def train_test_split(self):
        self.train_data, self.test_data = train_test_split(self.data, test_size=0.2, random_state=42)   
        
    def null_summary(self,df):
        print(df.isnull().sum())
        all_data_na = (df.isnull().sum() / len(df)) * 100
        all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
        missing_data.head()
        if missing_data.size> 0:
            f, ax = plt.subplots(figsize=(15, 12))
            plt.xticks(rotation='90')
            sns.barplot(x=all_data_na.index, y=all_data_na)
            plt.xlabel('Features', fontsize=15)
            plt.ylabel('Percent of missing values', fontsize=15)
            plt.title('Percent missing data by feature', fontsize=15)
            plt.show()
            
    def get_stats(self,df,variable):
        print("Skewness: %f" % df[variable].skew())
        print("Kurtosis: %f" % df[variable].kurt())
        print("Mean: %f" % df[variable].mean())
        print("Variance: %f" % (df[variable].var()))

            
    def correlation_map(self,df,outcome,k):
        corrmat = df.corr()
        col_large = corrmat.nlargest(k+1,outcome)[outcome].index
        col_small = corrmat.nsmallest(k,outcome)[outcome].index
        cols = col_large.union(col_small)
        cm = np.corrcoef(corrmat[cols].loc[cols].values.T)
        print(corrmat[cols].loc[outcome])
        #sns.set(font_scale=1.25)
        hm = sns.heatmap(cm, cbar=True, vmax=1,annot=True, fmt='.1f', annot_kws={'size': 8}, yticklabels=cols.values, xticklabels=cols.values)
        return cols
    
    def encode_organization(self,x):
            if 'Academy' in x:
                return 1
            elif 'Instituion' in x:
                return 2
            elif 'College' in x:
                return 3
            elif 'University' in x:
                return 4
            else:
                return 0
    
    def encode_cost(self,df):
        prices = []
        for i in range(len(df)):
            if pd.notnull(df['NPT4_PUB'][i]):
                prices.append(df['NPT4_PUB'][i])
            elif pd.notnull(df['NPT4_PRIV'][i]):
                prices.append(df['NPT4_PRIV'][i])
            else:
                prices.append(np.NaN) 
        
        df.loc[:,'NET_COST'] = pd.Series(prices, index=df.index)
        
        return df
    
    def encode_25KBinary(self, df):
        vals = []
        for i in range(len(df)):
            if float(df['gt_25k_p6'][i]) >= 0.5:
                vals.append(1)
            elif float(df['gt_25k_p6'][i]) < 0.5:
                vals.append(0)
            else:
                vals.append(np.NaN)
        
        df.loc[:,'Binary_25k'] = pd.Series(vals, index=df.index)
        
        return df
    
    def encode_state(self,x):
        west = ['WA','MT','OR','ID','WT','CA','NV','UT','CO','AZ','NM']
        midwest = ['ND','MN','SD','NE','KS','IA','MO','WI','IL','MI','IN','OH','WY']
        northeast = ['NY','PA','NJ','RI','CT','MA','VT','NH','ME']
        south = ['TX','OK','AR','LA','MS','AL','GA','TN','KY','WV','DC','MD','VA','DE','NC','SC','FL']
        others =['AK','HI','PR','PW','AS','GU','FM','VI']
        
        if x in others:
            return 0
        elif x in midwest:
            return 1
        elif x in south:
            return 2
        elif x in west:
            return 3
        elif x in northeast:
            return 4
        else: return x
        

In [40]:
project = Project()
project._load_data_('train.csv')

,Unnamed: 0,INSTNM,CITY,STABBR,PREDDEG,CONTROL,LOCALE,SATVRMID,SATMTMID,SATWRMID,...,RET_PTL4,PCTFLOAN,UG25abv,GRAD_DEBT_MDN_SUPP,GRAD_DEBT_MDN10YR_SUPP,RPY_3YR_RT_SUPP,C150_4_POOLED_SUPP,C200_L4_POOLED_SUPP,md_earn_wne_p10,gt_25k_p6
0,66,Trinity College of Florida,Trinity,FL,3,2,21.0,NaN,NaN,NaN,...,NaN,0.6557,0.4286,23000,255.34715,0.673076923,0.48001,NaN,31700,0.425287366
1,131,Hampton University,Hampton,VA,3,2,12.0,505.0,500.0,NaN,...,NaN,0.2978,0.1072,27474,305.0177217,0.678118393,0.634160079,NaN,42200,0.644176126
2,31,Empire Beauty School-Dixie,Louisville,KY,1,3,11.0,NaN,NaN,NaN,...,0.2000,0.7136,0.3902,8275,91.86946375,0.669099757,NaN,0.390740397,14800,0.224137932
3,84,Borner's Barber College,Los Angeles,CA,1,3,11.0,NaN,NaN,NaN,...,0.3000,0.0000,0.8261,NaN,NaN,PrivacySuppressed,NaN,PrivacySuppressed,NaN,NaN
4,289,ITT Technical Institute-Lake Mary,Lake Mary,FL,2,3,21.0,NaN,NaN,NaN,...,NaN,0.8343,0.8401,25833.5,286.8048087,0.394119686,0.35041453,NaN,38400,0.591855824
5,371,Hays Academy of Hair Design,Salina,KS,1,3,33.0,NaN,NaN,NaN,...,NaN,0.8182,0.3175,9833,109.1664577,0.674698795,NaN,NaN,NaN,NaN
6,333,American Academy of Dramatic Arts-Los Angeles,Los Angeles,CA,2,2,11.0,NaN,NaN,NaN,...,NaN,0.5701,0.1589,13240,146.991142,0.681818182,NaN,0.634754783,30200,0.416666657
7,153,Troy University-Phenix City Campus,Phenix City,AL,0,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26000,288.6533,0.543508214,NaN,NaN,36600,0.613958538
8,246,The College of Wooster,Wooster,OH,3,2,32.0,610.0,605.0,605.0,...,NaN,0.5144,0.0080,25500,283.102275,0.9125,0.753638017,NaN,44400,0.660766959
9,204,University of Baltimore,Baltimore,MD,3,1,11.0,480.0,460.0,455.0,...,NaN,0.5768,0.5085,20500,227.592025,0.684303351,0.3759,NaN,58000,0.824536383


In [43]:
df = pd.read_csv('train.csv')

In [44]:
df = project.data
df = df.drop(df.columns[0],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 42 columns):
INSTNM                    336 non-null object
CITY                      336 non-null object
STABBR                    336 non-null object
PREDDEG                   336 non-null int64
CONTROL                   336 non-null int64
LOCALE                    319 non-null float64
SATVRMID                  48 non-null float64
SATMTMID                  48 non-null float64
SATWRMID                  27 non-null float64
ACTCMMID                  44 non-null float64
ACTENMID                  38 non-null float64
ACTMTMID                  38 non-null float64
ACTWRMID                  13 non-null float64
SAT_AVG                   49 non-null float64
DISTANCEONLY              319 non-null float64
UGDS                      306 non-null float64
UGDS_WHITE                306 non-null float64
UGDS_BLACK                306 non-null float64
UGDS_HISP                 306 non-null float64
UGDS_ASIAN       

In [45]:
df = df.apply(lambda x: x.replace('PrivacySuppressed',np.NAN))
df = project.encode_cost(df)
df = project.encode_25KBinary(df)

In [46]:
df['RPY_3YR_RT_SUPP'] = pd.to_numeric(df['RPY_3YR_RT_SUPP'])
df['md_earn_wne_p10'] = pd.to_numeric(df['md_earn_wne_p10'])
df['GRAD_DEBT_MDN_SUPP'] = pd.to_numeric(df['GRAD_DEBT_MDN_SUPP'])
df['GRAD_DEBT_MDN10YR_SUPP'] = pd.to_numeric(df['GRAD_DEBT_MDN10YR_SUPP'])
df['C150_4_POOLED_SUPP'] = pd.to_numeric(df['C150_4_POOLED_SUPP'])
df['C200_L4_POOLED_SUPP'] = pd.to_numeric(df['C200_L4_POOLED_SUPP'])

In [47]:
columns = df.columns
#drop outcomes variables and "name" variables 
treatment_variables = columns.drop(['INSTNM','CITY','STABBR','RPY_3YR_RT_SUPP', 'md_earn_wne_p10', 'gt_25k_p6', 'Binary_25k'])

In [48]:
#only keep the variables if less than 20% of the values are missing ( SHOULD I KEEP IT at 20%?)
keep_variables = []
for i in range(len(treatment_variables)):
    x = df[str(treatment_variables[i])].isnull().sum()/float((len(df)))
    if x <= 0.2:
        keep_variables.append(treatment_variables[i]) 

In [49]:
keep_variables

['PREDDEG',
 'CONTROL',
 'LOCALE',
 'DISTANCEONLY',
 'UGDS',
 'UGDS_WHITE',
 'UGDS_BLACK',
 'UGDS_HISP',
 'UGDS_ASIAN',
 'UGDS_AIAN',
 'UGDS_NHPI',
 'UGDS_2MOR',
 'UGDS_NRA',
 'UGDS_UNKN',
 'PPTUG_EF',
 'PCTPELL',
 'PCTFLOAN',
 'UG25abv',
 'GRAD_DEBT_MDN_SUPP',
 'GRAD_DEBT_MDN10YR_SUPP',
 'NET_COST']

In [50]:
#correlations for RPY_3YR_RT_SUPP
for i in range(len(keep_variables)):
    print str(keep_variables[i])
    print df['RPY_3YR_RT_SUPP'].corr(df[str(keep_variables[i])])

PREDDEG
0.41195206057269423
CONTROL
-0.3261100801479304
LOCALE
0.08168255497972415
DISTANCEONLY
0.05069499664203906
UGDS
0.1760012325037092
UGDS_WHITE
0.46492732204439524
UGDS_BLACK
-0.4054922206913137
UGDS_HISP
-0.24925158539030579
UGDS_ASIAN
0.21773867892349655
UGDS_AIAN
-0.06694708969820343
UGDS_NHPI
-0.05706172562457254
UGDS_2MOR
0.03307421457283581
UGDS_NRA
0.2809839411391704
UGDS_UNKN
-0.06485775203603199
PPTUG_EF
-0.06273166447098115
PCTPELL
-0.5249647263345588
PCTFLOAN
-0.03015274997764476
UG25abv
-0.46985207003761786
GRAD_DEBT_MDN_SUPP
0.17582157858338796
GRAD_DEBT_MDN10YR_SUPP
0.17582157855883793
NET_COST
0.16511478937228596


In [51]:
#regressions for RPY_3YR_RT_SUPP
for i in range(len(keep_variables)):
    print (i)
    print (smf.ols('RPY_3YR_RT_SUPP ~ ' + str(keep_variables[i]), data=df).fit().summary())

0
                            OLS Regression Results                            
Dep. Variable:        RPY_3YR_RT_SUPP   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     54.78
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           1.75e-12
Time:                        22:32:21   Log-Likelihood:                 88.151
No. Observations:                 270   AIC:                            -172.3
Df Residuals:                     268   BIC:                            -165.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4430      0.022     20.345      0

                            OLS Regression Results                            
Dep. Variable:        RPY_3YR_RT_SUPP   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     21.60
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           5.42e-06
Time:                        22:32:22   Log-Likelihood:                 69.268
No. Observations:                 254   AIC:                            -134.5
Df Residuals:                     252   BIC:                            -127.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5705      0.012     46.373      0.0

In [52]:
#correlations for md_earn_wne_p10
for i in range(len(keep_variables)):
    print str(keep_variables[i])
    print df['md_earn_wne_p10'].corr(df[str(keep_variables[i])])

PREDDEG
0.4655136713650063
CONTROL
-0.15736524407428276
LOCALE
-0.041946584251139785
DISTANCEONLY
0.009427700774562528
UGDS
0.2513840066266706
UGDS_WHITE
0.19380623900842853
UGDS_BLACK
-0.1408290566352015
UGDS_HISP
-0.1937505670598538
UGDS_ASIAN
0.3324676445649233
UGDS_AIAN
-0.10540507785989933
UGDS_NHPI
0.03930500193618835
UGDS_2MOR
-0.0193596184968103
UGDS_NRA
0.31990139837371884
UGDS_UNKN
0.04523559217341784
PPTUG_EF
-0.006521897026611991
PCTPELL
-0.3820015587477176
PCTFLOAN
0.1808783740872359
UG25abv
-0.18441329942042267
GRAD_DEBT_MDN_SUPP
0.5688044013741512
GRAD_DEBT_MDN10YR_SUPP
0.5688044013712731
NET_COST
0.33818051379041075


In [53]:
#regressions for md_earn_wne_p10
for i in range(len(keep_variables)):
    print (i)
    #df[str(treatment_variables[i])]= df[str(treatment_variables[i])].fillna(np.nanmedian(df[str(treatment_variables[i])]))
    print (smf.ols('md_earn_wne_p10 ~ ' + str(keep_variables[i]), data=df).fit().summary())

0
                            OLS Regression Results                            
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     67.50
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           1.24e-14
Time:                        22:32:27   Log-Likelihood:                -2700.6
No. Observations:                 246   AIC:                             5405.
Df Residuals:                     244   BIC:                             5412.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.848e+04   1978.611      9.339      0

                            OLS Regression Results                            
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.4737
Date:                Mon, 12 Mar 2018   Prob (F-statistic):              0.492
Time:                        22:32:27   Log-Likelihood:                -2516.2
No. Observations:                 233   AIC:                             5036.
Df Residuals:                     231   BIC:                             5043.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.163e+04    869.316     36.380      0.0

In [54]:
#correlations for '25k_or_above'
for i in range(len(keep_variables)):
    print str(keep_variables[i])
    print df['Binary_25k'].corr(df[str(keep_variables[i])])

PREDDEG
0.39636765679645297
CONTROL
-0.15163807316048447
LOCALE
-0.10347275931411307
DISTANCEONLY
0.09335200560186736
UGDS
0.22831962129001293
UGDS_WHITE
0.15834698601115635
UGDS_BLACK
-0.13374892564831256
UGDS_HISP
-0.17474408408610814
UGDS_ASIAN
0.25590365506299945
UGDS_AIAN
-0.09082501386437898
UGDS_NHPI
0.01235225856116062
UGDS_2MOR
0.038289320500239185
UGDS_NRA
0.20061515281816797
UGDS_UNKN
0.10901886982730172
PPTUG_EF
0.045929278837877265
PCTPELL
-0.24810142876650657
PCTFLOAN
0.15628626229871867
UG25abv
-0.14087163166738353
GRAD_DEBT_MDN_SUPP
0.48149686747241194
GRAD_DEBT_MDN10YR_SUPP
0.4814968674866805
NET_COST
0.21972206160949767


In [64]:
#regressions for 25k_or_above
for i in range(len(keep_variables)):
    print (i)
    if i==3:
        next
    else:
        print (smf.logit('Binary_25k ~ ' + str(keep_variables[i]), data=df).fit().summary())

0
Optimization terminated successfully.
         Current function value: 0.610197
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:             Binary_25k   No. Observations:                  256
Model:                          Logit   Df Residuals:                      254
Method:                           MLE   Df Model:                            1
Date:                Mon, 12 Mar 2018   Pseudo R-squ.:                  0.1196
Time:                        22:37:46   Log-Likelihood:                -156.21
converged:                       True   LL-Null:                       -177.44
                                        LLR p-value:                 7.234e-11
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6951      0.308     -5.509      0.000      -2.298      -1.092
PREDDEG        0.9503      

                           Logit Regression Results                           
Dep. Variable:             Binary_25k   No. Observations:                  242
Model:                          Logit   Df Residuals:                      240
Method:                           MLE   Df Model:                            1
Date:                Mon, 12 Mar 2018   Pseudo R-squ.:                0.008840
Time:                        22:37:47   Log-Likelihood:                -166.05
converged:                       True   LL-Null:                       -167.53
                                        LLR p-value:                   0.08524
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1945      0.145     -1.342      0.180      -0.479       0.090
UGDS_UNKN      1.8596      1.141      1.630      0.103      -0.376       4.095
14
Optimization terminated successfully.
         Cu

In [56]:
total = ''
for i in range(len(keep_variables)):
    if i== len(keep_variables)-1:
        total = total + str(keep_variables[i])
    else:
        total = total + str(keep_variables[i]) + ' + '

In [57]:
total

'PREDDEG + CONTROL + LOCALE + DISTANCEONLY + UGDS + UGDS_WHITE + UGDS_BLACK + UGDS_HISP + UGDS_ASIAN + UGDS_AIAN + UGDS_NHPI + UGDS_2MOR + UGDS_NRA + UGDS_UNKN + PPTUG_EF + PCTPELL + PCTFLOAN + UG25abv + GRAD_DEBT_MDN_SUPP + GRAD_DEBT_MDN10YR_SUPP + NET_COST'

In [58]:
smf.ols('RPY_3YR_RT_SUPP ~ ' + total, data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        RPY_3YR_RT_SUPP   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     18.94
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           1.73e-37
Time:                        22:35:45   Log-Likelihood:                 181.00
No. Observations:                 231   AIC:                            -318.0
Df Residuals:                     209   BIC:                            -242.3
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 21.8916     94.519      0.232      0.817    -164.440     208.224
PREDDEG                    0.0584      0.018      3.180      0.002       0.022       0.095
CONTROL                   -0.0353      0.017     -2.029      0.044      -0.070      -0.001
LOCALE                    -0.0005      0.001     -0.470      0.639      -0.003       0.002
DISTANCEONLY               0.2427      0.165      1.472      0.142      -0.082       0.568
UGDS                   -2.482e-06   2.33e-06     -1.064      0.288   -7.08e-06    2.12e-06
UGDS_WHITE               -21.0725     94.518     -0.223      0.824    -207.404     165.259
UGDS_BLACK               -21.3463     94.515     -0.226      0.822    -207.671     164.979
UGDS_HISP                -21.2657     94.518     -0.225      0.822    -207.598     165.066
UGDS_ASIAN               -20.0767     94.539     -0.212      0.832    -206.448     166.295
UGDS_AIAN                -22.4227     94.458     -0.237      0.813    -208.635     163.790
UGDS_NHPI                -22.1860     94.364     -0.235      0.814    -208.214     163.842
UGDS_2MOR                -22.0597     94.531     -0.233      0.816    -208.417     164.297
UGDS_NRA                 -20.9580     94.523     -0.222      0.825    -207.299     165.383
UGDS_UNKN                -21.2722     94.519     -0.225      0.822    -207.606     165.061
PPTUG_EF                  -0.0463      0.046     -1.007      0.315      -0.137       0.044
PCTPELL                   -0.3469      0.076     -4.584      0.000      -0.496      -0.198
PCTFLOAN                   0.2738      0.065      4.241      0.000       0.147       0.401
UG25abv                   -0.1488      0.051     -2.898      0.004      -0.250      -0.048
GRAD_DEBT_MDN_SUPP      5972.2494   4627.484      1.291      0.198   -3150.278    1.51e+04
GRAD_DEBT_MDN10YR_SUPP -5.379e+05   4.17e+05     -1.291      0.198   -1.36e+06    2.84e+05
NET_COST                3.225e-06   1.74e-06      1.855      0.065   -2.02e-07    6.65e-06
==============================================================================
Omnibus:                        3.082   Durbin-Watson:                   2.174
Prob(Omnibus):                  0.214   Jarque-Bera (JB):                3.447
Skew:                           0.003   Prob(JB):                        0.178
Kurtosis:                       3.598   Cond. No.                     1.38e+12
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.77e-14. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [59]:
smf.ols('md_earn_wne_p10 ~ ' + total, data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.604
Method:                 Least Squares   F-statistic:                     16.16
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           2.62e-30
Time:                        22:35:54   Log-Likelihood:                -2047.3
No. Observations:                 200   AIC:                             4137.
Df Residuals:                     179   BIC:                             4206.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               1.382e+07   6.17e+06      2.241      0.026    1.65e+06     2.6e+07
PREDDEG                 1105.6898   1289.288      0.858      0.392   -1438.469    3649.849
CONTROL                -1660.7270   1249.397     -1.329      0.185   -4126.168     804.714
LOCALE                    98.7202     70.972      1.391      0.166     -41.329     238.769
DISTANCEONLY              -1.7790      1.891     -0.941      0.348      -5.511       1.953
UGDS                       0.1155      0.146      0.789      0.431      -0.173       0.405
UGDS_WHITE              -1.38e+07   6.17e+06     -2.237      0.027    -2.6e+07   -1.62e+06
UGDS_BLACK             -1.379e+07   6.17e+06     -2.236      0.027    -2.6e+07   -1.62e+06
UGDS_HISP              -1.379e+07   6.17e+06     -2.236      0.027    -2.6e+07   -1.62e+06
UGDS_ASIAN             -1.371e+07   6.17e+06     -2.223      0.027   -2.59e+07   -1.54e+06
UGDS_AIAN              -1.376e+07   6.16e+06     -2.232      0.027   -2.59e+07    -1.6e+06
UGDS_NHPI               -1.39e+07   6.16e+06     -2.257      0.025   -2.61e+07   -1.75e+06
UGDS_2MOR              -1.383e+07   6.17e+06     -2.241      0.026    -2.6e+07   -1.65e+06
UGDS_NRA               -1.377e+07   6.17e+06     -2.233      0.027   -2.59e+07    -1.6e+06
UGDS_UNKN              -1.379e+07   6.17e+06     -2.236      0.027    -2.6e+07   -1.62e+06
PPTUG_EF                1033.7604   3051.952      0.339      0.735   -4988.673    7056.194
PCTPELL                -2.593e+04   5130.772     -5.054      0.000   -3.61e+04   -1.58e+04
PCTFLOAN                1.169e+04   4286.212      2.728      0.007    3234.921    2.02e+04
UG25abv                 3888.1539   3537.171      1.099      0.273   -3091.765    1.09e+04
GRAD_DEBT_MDN_SUPP     -2.843e+08   3.02e+08     -0.941      0.348   -8.81e+08    3.12e+08
GRAD_DEBT_MDN10YR_SUPP  2.561e+10   2.72e+10      0.941      0.348   -2.81e+10    7.93e+10
NET_COST                   0.0633      0.117      0.540      0.590      -0.168       0.295
==============================================================================
Omnibus:                       43.903   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              121.304
Skew:                           0.915   Prob(JB):                     4.56e-27
Kurtosis:                       6.348   Cond. No.                     3.25e+20
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.25e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [62]:
repay_pvalues = smf.ols('RPY_3YR_RT_SUPP ~ ' + total, data=df).fit().pvalues
smr.sandbox.stats.multicomp.multipletests(repay_pvalues, alpha=0.05, method='bonferroni')

(array([False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,
        False, False, False, False]),
 array([1.00000000e+00, 3.72974523e-02, 9.61414394e-01, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.72335291e-04, 7.36434578e-04, 9.13094129e-02, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00]),
 0.0023287975150316775,
 0.002272727272727273)

In [63]:
median_pvalues = smf.ols('md_earn_wne_p10 ~ ' + total, data=df).fit().pvalues
smr.sandbox.stats.multicomp.multipletests(median_pvalues, alpha=0.05, method='bonferroni')

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False, False, False, False]),
 array([5.78251325e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 5.83819863e-01, 5.84736189e-01,
        5.84459935e-01, 6.04946384e-01, 5.90467240e-01, 5.54752943e-01,
        5.77642812e-01, 5.89710101e-01, 5.84603484e-01, 1.00000000e+00,
        2.33741530e-05, 1.54139539e-01, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00]),
 0.0023287975150316775,
 0.002272727272727273)

In [65]:
#somehow DISTANCEONLY is causing an error
total = ''
for i in range(len(keep_variables)):
    if i== len(keep_variables)-1:
        total = total + str(keep_variables[i])
    elif str(keep_variables[i])== 'DISTANCEONLY':
             next
    else:
        total = total + str(keep_variables[i]) + ' + '

In [66]:
smf.logit('Binary_25k ~' + total , data=df).fit().summary()

         Current function value: 12.015554
         Iterations: 35


/Users/marika/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Binary_25k   No. Observations:                  206
Model:                          Logit   Df Residuals:                      185
Method:                           MLE   Df Model:                           20
Date:                Mon, 12 Mar 2018   Pseudo R-squ.:                  -16.34
Time:                        22:38:55   Log-Likelihood:                -2475.2
converged:                      False   LL-Null:                       -142.78
                                        LLR p-value:                     1.000
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               6.293e+05   4.62e+05      1.363      0.173   -2.75e+05    1.53e+06
PREDDEG                   15.0872     10.209      1.478      0.139      -4.921      35.096
CONTROL                   20.8405     15.636      1.333      0.183      -9.806      51.487
LOCALE                     0.3421      0.298      1.150      0.250      -0.241       0.925
UGDS                       0.0018      0.002      1.134      0.257      -0.001       0.005
UGDS_WHITE             -6.294e+05   4.62e+05     -1.363      0.173   -1.53e+06    2.75e+05
UGDS_BLACK             -6.294e+05   4.62e+05     -1.363      0.173   -1.53e+06    2.75e+05
UGDS_HISP              -6.294e+05   4.62e+05     -1.363      0.173   -1.53e+06    2.75e+05
UGDS_ASIAN             -6.287e+05   4.61e+05     -1.363      0.173   -1.53e+06    2.75e+05
UGDS_AIAN              -6.293e+05   4.62e+05     -1.363      0.173   -1.53e+06    2.75e+05
UGDS_NHPI              -6.303e+05   4.62e+05     -1.364      0.173   -1.54e+06    2.76e+05
UGDS_2MOR              -6.297e+05   4.62e+05     -1.364      0.173   -1.53e+06    2.75e+05
UGDS_NRA               -6.293e+05   4.61e+05     -1.364      0.173   -1.53e+06    2.75e+05
UGDS_UNKN              -6.293e+05   4.62e+05     -1.363      0.173   -1.53e+06    2.75e+05
PPTUG_EF                  51.5418     36.497      1.412      0.158     -19.991     123.074
PCTPELL                 -190.7661    142.067     -1.343      0.179    -469.213      87.680
PCTFLOAN                 104.9458     79.010      1.328      0.184     -49.912     259.803
UG25abv                  -33.3121     21.339     -1.561      0.118     -75.135       8.511
GRAD_DEBT_MDN_SUPP         3.3118   2467.127      0.001      0.999   -4832.168    4838.792
GRAD_DEBT_MDN10YR_SUPP  -298.0481   2.22e+05     -0.001      0.999   -4.36e+05    4.35e+05
NET_COST                  -0.0022      0.002     -1.301      0.193      -0.006       0.001
==========================================================================================

Possibly complete quasi-separation: A fraction 0.64 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""